### **Database Monitoring (Google Sheets) - Cocaine Seizures 2025**
#### InSight Crime - MAD Unit. 
June, 2025

##### Luis Felipe Villota Macías

---------------------



### 1. Goals

* MONITOR & VALIDATE -> data in a shared Google Sheet using automated checks to ensure accuracy, consistency and data quality.

* HIGHLIGHT & CORRECT -> signal invalid entries and intervene directly in the sheet. 

* REPORT -> Generate weekly (every Friday) reports to support governance. 

### Why?
- Good practice for reproducibility, transparency and scalability (can be applied to other relevant dbs: e.g., homicides, etc).  
- Google Sheets validation is basic -> we can enforce custom rules (e.g., logical dates, quantities, relationships, normalize names, etc).
- Facilitates reporting and knowledge sharing.
- Overall: provides quality data ready to be analyzed

### 2. Project Setup

#### 2.1 Version Control

I decided to create a single GitHub repository ([FelipeVillota/db-check-cocaine-seizures](https://github.com/FelipeVillota/db-check-cocaine-seizures)). I keep the repository `private` with the possibility to give access to the online repo at any time. 

#### 2.2 Reproducible Environment

In [28]:
# IMPORTANT
# To create venv
# python -m venv venv-db-watch

# To activate environment, run in Terminal:
# # (optional, temporary auth) 
# Set-ExecutionPolicy -Scope Process -ExecutionPolicy Bypass 
# venv-db-watch\Scripts\activate

# Then select respective kernel --> also install ipykernel package to connect to kernel

# Update list master list
# pip freeze > requirements.txt

In [29]:
# Checking venv-db-watch works
import sys
print(sys.executable)

c:\Users\USER\Desktop\ic\db-check-cocaine-seizures\venv-db-watch\Scripts\python.exe


#### 2.3 Loading Libraries

In [30]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas

In [31]:
import os
import re
import requests
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread
from google.oauth2.service_account import Credentials
from gspread_formatting import format_cell_ranges, CellFormat, Color

# pip freeze > requirements.txt

### 3.  Approach

The general idea is to create a modular client (frontend) call that is able to extract the desired subset of data from the API server (backend); -and, make it easily reusable for future queries.


### 4. Execution

#### 4.1 Accessing the API

In [32]:
# Path to downloaded JSON credentials
SERVICE_ACCOUNT_FILE = 'C:/Users/USER/Desktop/ic/llavero/summer-sector-439022-v6-2eafffbbfb90.json'

In [33]:
# ID of the Google Sheet (from the URL)

original_id = '1t61MafCmnRe2QN082Bk1V0IxBSIW8UUqH1g5mULgb2o'
test_id = '18hdnhuqvH4vdXuL16CBI7BpaMCu8K81T2NThbK6OJzk'

SPREADSHEET_ID = test_id
SPREADSHEET_ID

'18hdnhuqvH4vdXuL16CBI7BpaMCu8K81T2NThbK6OJzk'

In [34]:

# Range to read from your sheet (e.g. 'Sheet1!A1:Z1000')
RANGE_NAME = '2025!A1:Z10000'  # Adjust the range as needed

In [35]:

# Define scopes for Google Sheets and Drive API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

In [36]:
#Authenticate and build the service
creds = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API to read data
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])

# Convert to DataFrame for easier manipulation
df = pd.DataFrame(values[1:], columns=values[0])
print(df.head()) 

      Type   Time unit        Date Date 2  Year Month Day Duration  \
0  Seizure  Individual  2025-03-24         2025     3  24            
1  Seizure  Individual  2025-03-23         2025     3  23            
2  Seizure  Individual  2025-03-22         2025     3  22            
3  Seizure  Individual  2025-03-22         2025     3  22            
4  Seizure  Individual  2025-03-19         2025     3  19            

                       Type Drugs Quantity  ... Department/State  \
0                         Cocaine       10  ...                    
1                         Cocaine      2.5  ...                    
2                         Cocaine     2619  ...            Zulia   
3  Other (explain in Description)     1240  ...            Zulia   
4                         Cocaine    16.05  ...            Texas   

          Municipality/Port Origin country         Origin Area  \
0                  Florence                                      
1             Santo Domingo           

#### 4.2 Data Validation Management

✅  Implementation


1. Define rules per column

2. Create a function per rule block

3. Apply them row-wise, collecting violations

4. Mark validation status per row

5. (Optional) Write to a new column or Sheet tab

6. (Optional) Use Google Sheets API to apply color highlighting

In [37]:
# Info about the variables in the DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Type                             378 non-null    object
 1   Time unit                        378 non-null    object
 2   Date                             378 non-null    object
 3   Date 2                           378 non-null    object
 4   Year                             378 non-null    object
 5   Month                            378 non-null    object
 6   Day                              378 non-null    object
 7   Duration                         378 non-null    object
 8   Type Drugs                       378 non-null    object
 9   Quantity                         378 non-null    object
 10  Weight unit                      378 non-null    object
 11  seizure_kgs                      378 non-null    object
 12  Modus Operandi/place of seizure  378

In [38]:
# Unique values in each column 
for col in df.columns:
    print(f"{col} → {df[col].dropna().unique()}")

Type → ['Seizure']
Time unit → ['Individual' 'Month' 'Multi-Month' 'Other (explain in Description)'
 'Year']
Date → ['2025-03-24' '2025-03-23' '2025-03-22' '2025-03-19' '2025-03-25'
 '2025-03-26' '2025-03-27' '2025-03-20' '2025-03-10' '2025-03-21'
 '2025-03-30' '2025-03-29' '2025-01-01' '2025-03-28' '2025-03-31'
 '2025-04-01' '2025-04-02' '2025-04-03' '2025-04-04' '2025-04-07'
 '2025-04-08' '2025-04-09' '2025-04-06' '2025-04-05' '2025-04-13'
 '2025-04-12' '2025-04-14' '2025-04-10' '2025-02-02' '2025-04-15'
 '2025-04' '2025-04-16' '2025-04-17' '2025-04-11' '2025-04-18'
 '2025-04-21' '2025-04-20' '2025-04-22' '2025-04-19' '2025-04-23'
 '2025-04-24' '2025-04-25' '2025-02-01' '2025-04-26' '2025-02-13'
 '2025-04-29' '2025-04-30' '2025-05-03' '2025-05-05' '2025-05-06'
 '2025-05-07' '2025-05-02' '2025-05-09' '2025-04-28' '2025-05-08'
 '2025-05-04' '' '2025-05-11' '2025-05-12' '2025-05-13' '2025-05-10'
 '2025-05-15' '2025-05-14' '2025-05-17' '2025-04-27' '2025-05-18'
 '2025-05-19' '2025-05-16'

In [40]:
# Update column name directly in Google Sheets
def update_column_name(old_name, new_name):
    # Get headers
    headers = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range='2025!1:1').execute()['values'][0]
    
    # Find and update column
    if old_name in headers:
        col_index = headers.index(old_name)
        col_letter = chr(65 + col_index)  # Convert to A, B, C...
        
        # Update in sheet
        sheet.values().update(
            spreadsheetId=SPREADSHEET_ID,
            range=f'2025!{col_letter}1',
            valueInputOption='RAW',
            body={'values': [[new_name]]}
        ).execute()
        
        print(f"✅ '{old_name}' → '{new_name}'")
    else:
        print(f"❌ '{old_name}' not found")

# Usage
update_column_name('Date-test', 'Date')

# Refresh DataFrame
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])
df = pd.DataFrame(values[1:], columns=values[0])

✅ 'Date-test' → 'Date'


In [24]:
# Format date column to DD-MM-YYYY directly in Google Sheets
def format_dates_to_dd_mm_yyyy(date_column_name='Date'):
    # Get all data
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print("❌ No data found")
        return
    
    headers = values[0]
    if date_column_name not in headers:
        print(f"❌ '{date_column_name}' column not found")
        return
    
    col_index = headers.index(date_column_name)
    col_letter = chr(65 + col_index)
    
    # Process dates
    updated_values = []
    for i, row in enumerate(values[1:], 2):  # Start from row 2
        if col_index < len(row) and row[col_index]:
            try:
                # Parse date (assuming YYYY-MM-DD format)
                date_obj = datetime.strptime(row[col_index], "%Y-%m-%d")
                # Format to DD-MM-YYYY
                formatted_date = date_obj.strftime("%d-%m-%Y")
                updated_values.append([formatted_date])
            except ValueError:
                # Keep original if can't parse
                updated_values.append([row[col_index]])
        else:
            updated_values.append([''])
    
    # Update the entire date column
    if updated_values:
        sheet.values().update(
            spreadsheetId=SPREADSHEET_ID,
            range=f'2025!{col_letter}2:{col_letter}{len(updated_values)+1}',
            valueInputOption='RAW',
            body={'values': updated_values}
        ).execute()
        
        print(f"✅ Dates formatted to DD-MM-YYYY in column {col_letter}")

# Usage
format_dates_to_dd_mm_yyyy('Date')  # or whatever your date column is called

✅ Dates formatted to DD-MM-YYYY in column C


In [44]:
def enforce_time_unit_rule():
    # Get all data
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print("❌ No data found")
        return
    
    headers = values[0]
    
    # Find required columns
    try:
        time_unit_col = headers.index('Time unit')
        date2_col = headers.index('Date 2')
    except ValueError:
        print("❌ Required columns not found (Time unit and/or Date 2)")
        return
    
    # Prepare batch update requests
    requests = []
    violations = []
    
    for i, row in enumerate(values[1:], 2):  # Start from row 2 (1-indexed)
        # Check if row has enough columns
        if len(row) <= max(time_unit_col, date2_col):
            continue
            
        time_unit = row[time_unit_col].strip().lower() if time_unit_col < len(row) and row[time_unit_col] else None
        date2 = row[date2_col].strip() if date2_col < len(row) and row[date2_col] else None
        
        if time_unit == 'individual' and date2:
            violations.append(i)
            
            # Add request to clear the cell
            requests.append({
                'repeatCell': {
                    'range': {
                        'sheetId': SHEET_ID,  # You need to define this
                        'startRowIndex': i-1,  # 0-based
                        'endRowIndex': i,
                        'startColumnIndex': date2_col,
                        'endColumnIndex': date2_col+1
                    },
                    'cell': {
                        'userEnteredValue': {'stringValue': ''}
                    },
                    'fields': 'userEnteredValue'
                }
            })
            
            # Add request to format cell red
            requests.append({
                'repeatCell': {
                    'range': {
                        'sheetId': SHEET_ID,
                        'startRowIndex': i-1,
                        'endRowIndex': i,
                        'startColumnIndex': date2_col,
                        'endColumnIndex': date2_col+1
                    },
                    'cell': {
                        'userEnteredFormat': {
                            'backgroundColor': {
                                'red': 1,
                                'green': 0.8,
                                'blue': 0.8
                            }
                        }
                    },
                    'fields': 'userEnteredFormat.backgroundColor'
                }
            })
    
    # Execute batch update if there are violations
    if requests:
        body = {
            'requests': requests
        }
        sheet.batchUpdate(spreadsheetId=SPREADSHEET_ID, body=body).execute()
        print(f"✅ Rule enforced: {len(violations)} violations found and corrected")
        print(f"   Rows with violations: {violations}")
    else:
        print("✅ No violations found")

In [ ]:
# ✅ GATHER DROPDOWN OPTIONS FOR EACH COLUMN
dropdown_options = {}

for col in df.columns:
    unique_values = df[col].dropna().unique().tolist()
    dropdown_options[col] = unique_values

# Optional: Print a preview of the dropdown dictionary
for col, options in dropdown_options.items():
    print(f"{col}: {options[:10]}{'...' if len(options) > 10 else ''}")


{}

#### 4.3 Data Anlysis & Reporting
